In [ ]:
import os
import re
import hashlib
import pandas as pd
from html import unescape
# from sqlalchemy import create_engine

raw_path = r"C:\Users\JKK4V3PX\healthcare_job_analyze\healthcare_job\data\emed_careers_eu.csv"
df = pd.read_csv(raw_path)

In [2]:
# Cell 2: 读取 CSV 并做初步检查（运行后看输出）

print("Rows, Columns:", df.shape)
display(df.head(5))
print("--- Info ---")
print(df.info())
print("\n--- Missing per column ---")
print(df.isnull().sum().sort_values(ascending=False))


Rows, Columns: (39774, 8)


,category,company_name,job_description,job_title,job_type,location,post_date,salary_offered
0,Clinical Research,PPD GLOBAL LTD,"As part of our on-going growth, we are current...",Senior / Medical Writer (Regulatory),Permanent,Cambridge,4/14/2018,Competitive
1,Science,AL Solutions,Manager of Biometrics – Italy\nAL Solutions ar...,Manager of Biometrics,Permanent,Europe,4/16/2018,NaN
2,Science,Seltek Consultants Ltd,A fantastic opportunity has arisen for an expe...,Field Service Engineer | Chromatography,Permanent,UK,4/16/2018,NaN
3,Data Management and Statistics,Docs International UK Limited,Job Details\n:\nUtilise extensive clinical dat...,Data Manager of Project Management,Permanent,M4 Corridor,4/11/2018,On Application
4,Science,Hyper Recruitment Solutions Ltd,Hyper Recruitment Solutions are currently look...,Strategic Market Analyst,Permanent,Cambridge,4/13/2018,NaN


--- Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39774 entries, 0 to 39773
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category         30000 non-null  object
 1   company_name     30000 non-null  object
 2   job_description  30000 non-null  object
 3   job_title        30000 non-null  object
 4   job_type         30000 non-null  object
 5   location         30000 non-null  object
 6   post_date        30000 non-null  object
 7   salary_offered   22685 non-null  object
dtypes: object(8)
memory usage: 2.4+ MB
None

--- Missing per column ---
salary_offered     17089
category            9774
job_description     9774
company_name        9774
job_title           9774
job_type            9774
location            9774
post_date           9774
dtype: int64


In [3]:
# Cell 3: 定义若干辅助函数，用于后续清理数据

# define keywords for searching
SKILL_KEYWORDS = [
    "python","sql","sas","r","cdisc","adverse","pharmacovigilance","spss",
    "machine learning","data analysis","biostatistics","clinical","gcp","cdash"
]

# 清理字符串的前后空格,并把空/NaN标准化为 None
def safe_strip(x):
    if pd.isna(x):
        return None
    s = str(x)
    s = s.strip()
    return s if s != "" else None


# 日期转换
def parse_date(s):
    try:
        return pd.to_datetime(s, errors='coerce', dayfirst=True)
    except:
        return pd.NaT


# 把location拆分成地区和国家
def parse_location(loc):
    # 依据逗号或竖线或短横拆分：最后一段通常为country
    if pd.isna(loc):
        return (None, None)
    s = str(loc).strip()
    parts = [p.strip() for p in re.split(r',|\||-', s) if p.strip()]
    if len(parts) == 0:
        return (None, None)
    if len(parts) == 1:
        # 只有一个字段：可能是 "Europe" 或 "UK" 或 "Cambridge"
        # 简单规则：若单词长度<=3并全大写，可能是国家缩写 -> 视为 country 否则 city
        token = parts[0]
        if token.isupper() and len(token) <= 3:
            return (None, token)
        # 若是常见 continent/country词
        if token.lower() in ["europe","uk","usa","us","united kingdom","germany","france"]:
            return (None, token)
        return (token, None)
    # 多段：最后一段当作 country，其余合并为 city
    city = ", ".join(parts[:-1])
    country = parts[-1]
    return (city if city else None, country if country else None)

#拆分薪资
def parse_salary(text):
    # 返回 (min, max, currency, parsed_flag)
    if pd.isna(text):
        return (None, None, None, False)
    s = str(text).strip()
    s_lower = s.lower()
    # 常见非结构化文本，判断特殊词
    if any(term in s_lower for term in ["competitive","on application","negotiable","not disclosed","tbd"]):
        return (None, None, None, False)
    # 找货币符号或货币文本
    cur = None
    cur_search = re.search(r'€|£|\$|usd|eur|gbp', s, flags=re.IGNORECASE)
    if cur_search:
        cur = cur_search.group(0)
    # 提取所有数字串（忽略小数点分隔）
    nums = re.findall(r'\d{3,}', s.replace(',', ''))
    if len(nums) == 0:
        # 尝试找较短数字如 50k, 45k
        k_match = re.findall(r'(\d{2,3})k', s_lower)
        if k_match:
            val = int(k_match[0]) * 1000
            return (val, val, cur, True)
        return (None, None, cur, False)
    if len(nums) == 1:
        v = int(nums[0])
        return (v, v, cur, True)
    # 2个以上，取前两个为区间
    vmin = int(nums[0])
    vmax = int(nums[1])
    if vmin > vmax:
        vmin, vmax = vmax, vmin
    return (vmin, vmax, cur, True)


# 去除 HTML 标签、转义符，归一化空白，返回清洁的描述字符串。避免描述里有换行或 HTML 导致展示/统计问题。
def clean_description(text):
    if pd.isna(text):
        return ""
    t = unescape(str(text))
    # 去掉HTML标签
    t = re.sub(r'<[^>]+>', ' ', t)
    # 转换连续空白为单空格
    t = re.sub(r'\s+', ' ', t).strip()
    return t


#把描述转小写，检查关键词列表，生成一个简单的技能标签，用于后续筛选或可视化
def extract_skill_flags(text):
    t = (text or "").lower()
    found = [kw for kw in SKILL_KEYWORDS if kw in t]
    return ",".join(found) if found else None


# 生成job id
def make_job_id(row):
    # 用 company+title+date 做hash，确保稳定
    key = f"{row.get('company_name','') or ''}|{row.get('job_title','') or ''}|{row.get('post_date_parsed_str','') or ''}"
    return hashlib.md5(key.encode('utf-8')).hexdigest()


In [4]:
# Cell 4: 实际清洗

# 1. 复制一份操作，保留原始 df
df_clean = df.copy()

# 2. 统一缺失值格式并消除无意义的空格
for c in ['category','company_name','job_description','job_title','job_type','location','post_date','salary_offered']:
    if c in df_clean.columns:
        df_clean[c] = df_clean[c].apply(safe_strip)

# 3. 去重
before = df_clean.shape[0]
df_clean = df_clean.drop_duplicates()
after = df_clean.shape[0]
print(f"Removed exact duplicates: {before - after}")

# 4. 解析日期
if 'post_date' in df_clean.columns:
    df_clean['post_date_parsed'] = df_clean['post_date'].apply(parse_date)
    # 也保留一个字符串格式（便于做id）
    df_clean['post_date_parsed_str'] = df_clean['post_date_parsed'].dt.strftime('%Y-%m-%d')

# 5. 解析location
if 'location' in df_clean.columns:
    loc_parsed = df_clean['location'].apply(parse_location)
    df_clean['city'] = loc_parsed.apply(lambda x: x[0])
    df_clean['country'] = loc_parsed.apply(lambda x: x[1])

# 6. 标准化job_type
def norm_job_type(s):
    if not s: return "Unknown"
    s = s.lower()
    if 'perm' in s: return 'Permanent'
    if 'contract' in s: return 'Contract'
    if 'temp' in s or 'temporary' in s: return 'Temporary'
    if 'intern' in s or 'graduate' in s: return 'Internship'
    return 'Other'
if 'job_type' in df_clean.columns:
    df_clean['job_type_std'] = df_clean['job_type'].apply(norm_job_type)

# 7. 清理job_description
if 'job_description' in df_clean.columns:
    df_clean['job_description_clean'] = df_clean['job_description'].apply(clean_description)
    df_clean['desc_word_count'] = df_clean['job_description_clean'].apply(lambda t: len(t.split()) if t else 0)
    df_clean['skill_flags'] = df_clean['job_description_clean'].apply(extract_skill_flags)

# 8. 解析salary
if 'salary_offered' in df_clean.columns:
    sal_parsed = df_clean['salary_offered'].apply(parse_salary)
    df_clean['salary_min'] = sal_parsed.apply(lambda x: x[0])
    df_clean['salary_max'] = sal_parsed.apply(lambda x: x[1])
    df_clean['salary_currency'] = sal_parsed.apply(lambda x: x[2])
    df_clean['salary_parsed'] = sal_parsed.apply(lambda x: x[3])

# 9. 生成 job_id 并根据 job_id 去重
df_clean['job_id'] = df_clean.apply(make_job_id, axis=1)
before = df_clean.shape[0]
df_clean = df_clean.drop_duplicates(subset=['job_id'])
after = df_clean.shape[0]
print(f"Removed duplicates by job_id: {before - after}")


Removed exact duplicates: 23022


C:\Users\JKK4V3PX\AppData\Local\Temp\ipykernel_19312\3506752157.py:21: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(s, errors='coerce', dayfirst=True)


Removed duplicates by job_id: 13108


In [ ]:
# 查看清洗后的信息
display(df_clean.head(10))

,category,company_name,job_description,job_title,job_type,location,post_date,salary_offered,post_date_parsed,post_date_parsed_str,...,country,job_type_std,job_description_clean,desc_word_count,skill_flags,salary_min,salary_max,salary_currency,salary_parsed,job_id
0,Clinical Research,PPD GLOBAL LTD,"As part of our on-going growth, we are current...",Senior / Medical Writer (Regulatory),Permanent,Cambridge,4/14/2018,Competitive,2018-04-14,2018-04-14,...,None,Permanent,"As part of our on-going growth, we are current...",359,"r,biostatistics,clinical",NaN,NaN,None,False,15cc819a5d9a4b298bcc5fd839bd65a3
1,Science,AL Solutions,Manager of Biometrics – Italy\nAL Solutions ar...,Manager of Biometrics,Permanent,Europe,4/16/2018,None,2018-04-16,2018-04-16,...,Europe,Permanent,Manager of Biometrics – Italy AL Solutions are...,291,"sas,r,clinical",NaN,NaN,None,False,31c1a5eac509952eeb8cfdccbeb2fcc4
2,Science,Seltek Consultants Ltd,A fantastic opportunity has arisen for an expe...,Field Service Engineer | Chromatography,Permanent,UK,4/16/2018,None,2018-04-16,2018-04-16,...,UK,Permanent,A fantastic opportunity has arisen for an expe...,374,r,NaN,NaN,None,False,7b6abf608611f7c6b97be78b5914d26c
3,Data Management and Statistics,Docs International UK Limited,Job Details\n:\nUtilise extensive clinical dat...,Data Manager of Project Management,Permanent,M4 Corridor,4/11/2018,On Application,2018-11-04,2018-11-04,...,None,Permanent,Job Details : Utilise extensive clinical data ...,279,"r,pharmacovigilance,clinical",NaN,NaN,None,False,63542ffb0393034e35a3f9753b274b73
4,Science,Hyper Recruitment Solutions Ltd,Hyper Recruitment Solutions are currently look...,Strategic Market Analyst,Permanent,Cambridge,4/13/2018,None,2018-04-13,2018-04-13,...,None,Permanent,Hyper Recruitment Solutions are currently look...,210,r,NaN,NaN,None,False,7c9de23d13d4dd71f917385947fe5b81
5,Manufacturing & Operations,BCF Recruitment Ltd,My client is currently recruiting for a Senior...,Senior Mechanical Design Engineer,Permanent,South East,4/10/2018,None,2018-10-04,2018-10-04,...,None,Permanent,My client is currently recruiting for a Senior...,336,r,NaN,NaN,None,False,369628bdd3a511d652223dae4c2ba9ff
6,Quality-assurance,Austin Fraser,Austin Fraser is proud to be supporting one of...,Head of Quality Excellence,Permanent,Europe,4/12/2018,"Car allowance, Pension, Bonus, Healthcare etc",2018-12-04,2018-12-04,...,Europe,Permanent,Austin Fraser is proud to be supporting one of...,543,"r,clinical",NaN,NaN,None,False,be5771562626f9f10d0e18acd579c013
7,Manufacturing & Operations,Hyper Recruitment Solutions Ltd,We are currently looking for a Quality Assuran...,Quality Assurance (QA) Document Controller,Temporary/Seasonal,South East,4/14/2018,None,2018-04-14,2018-04-14,...,None,Temporary,We are currently looking for a Quality Assuran...,319,r,NaN,NaN,None,False,7b8ccd6e99b94c17377d0e158cdb9052
8,Clinical Research,Iqvia Ltd.,Due to growth we have exciting opportunities w...,Clinical Data Team Lead,Permanent,Manchester,4/10/2018,Competitive + package,2018-10-04,2018-10-04,...,None,Permanent,Due to growth we have exciting opportunities w...,849,"r,clinical,gcp",NaN,NaN,None,False,a79f103d858ce81de2d5f01f5ee2ac8f
9,Medical Information and Pharmacovigilance,Barrington James Ltd,I am currently partnered with a rapidly growin...,SENIOR HEALTH ECONOMIST,Permanent,UK,4/10/2018,Competitive Package,2018-10-04,2018-10-04,...,UK,Permanent,I am currently partnered with a rapidly growin...,327,"r,biostatistics",NaN,NaN,None,False,2e50da2959c0690e4fbcefedab701823


          city country post_date_parsed job_type_std  salary_min  salary_max  \
0    Cambridge    None       2018-04-14    Permanent         NaN         NaN   
1         None  Europe       2018-04-16    Permanent         NaN         NaN   
2         None      UK       2018-04-16    Permanent         NaN         NaN   
3  M4 Corridor    None       2018-11-04    Permanent         NaN         NaN   
4    Cambridge    None       2018-04-13    Permanent         NaN         NaN   

   salary_parsed  
0          False  
1          False  
2          False  
3          False  
4          False  


In [6]:
clean_path = "cleaned_healthcare_jobs.csv"
df_clean.to_csv(clean_path, index=False)
print(f"Cleaned data saved to {clean_path}")

Cleaned data saved to cleaned_healthcare_jobs.csv


In [8]:
import os
from sqlalchemy import create_engine

# 输出目录
OUT_DIR = "data/dwh"
os.makedirs(OUT_DIR, exist_ok=True)

df = df_clean.copy()

# ---------- 1) 生成 dim_company ----------
if 'company_name' in df.columns:
    dim_company = df[['company_name']].drop_duplicates().reset_index(drop=True)
    dim_company['company_id'] = dim_company.index + 1
    # 规范列顺序
    dim_company = dim_company[['company_id','company_name']]
else:
    dim_company = pd.DataFrame(columns=['company_id','company_name'])

print("dim_company rows:", len(dim_company))

# ---------- 2) 生成 dim_job_title ----------
if 'job_title' in df.columns:
    dim_job_title = df[['job_title']].drop_duplicates().reset_index(drop=True)
    dim_job_title['job_title_id'] = dim_job_title.index + 1
    dim_job_title = dim_job_title[['job_title_id','job_title']]
else:
    dim_job_title = pd.DataFrame(columns=['job_title_id','job_title'])

print("dim_job_title rows:", len(dim_job_title))

# ---------- 3) 生成 dim_salary ----------
# 使用 salary_min, salary_max, salary_currency, salary_parsed 组合去重
salary_cols = [c for c in ['salary_min','salary_max','salary_currency','salary_parsed'] if c in df.columns]
if len(salary_cols) > 0:
    dim_salary = df[salary_cols].drop_duplicates().reset_index(drop=True)
    # 产生 id
    dim_salary['salary_id'] = dim_salary.index + 1
    # 把 id 放到最前面
    cols_order = ['salary_id'] + salary_cols
    dim_salary = dim_salary[cols_order]
else:
    dim_salary = pd.DataFrame(columns=['salary_id','salary_min','salary_max','salary_currency','salary_parsed'])

print("dim_salary rows:", len(dim_salary))

# ---------- 4) 把维度 id 合并回主表（fact 前的准备） ----------
# 复制一份避免改动原始 df_clean
fact_df = df.copy()

# 合并 company_id
if 'company_name' in fact_df.columns and not dim_company.empty:
    fact_df = fact_df.merge(dim_company, on='company_name', how='left')

# 合并 job_title_id
if 'job_title' in fact_df.columns and not dim_job_title.empty:
    fact_df = fact_df.merge(dim_job_title, on='job_title', how='left')

# 合并 salary_id (基于所有 salary_cols)
if len(salary_cols) > 0 and not dim_salary.empty:
    # 合并基于所有 salary_cols — 为安全构建临时 key
    # left_on tuple works by passing list of columns
    fact_df = fact_df.merge(dim_salary, on=salary_cols, how='left')
else:
    fact_df['salary_id'] = None

# ---------- 5) 为了保持一致，检查是否有缺失的 id（理论上不应该有） ----------
missing_company = fact_df['company_id'].isnull().sum() if 'company_id' in fact_df.columns else 0
missing_title = fact_df['job_title_id'].isnull().sum() if 'job_title_id' in fact_df.columns else 0
missing_salary = fact_df['salary_id'].isnull().sum() if 'salary_id' in fact_df.columns else 0
print(f"Missing ids -> company: {missing_company}, job_title: {missing_title}, salary: {missing_salary}")


# ---------- 6) 从 fact 表中移除重复列 ----------
drop_cols = ['post_date','post_date_parsed','job_description','job_type','location']
existing_drop = [c for c in drop_cols if c in fact_df.columns]
print("Dropping columns from fact:", existing_drop)
fact_df = fact_df.drop(columns=existing_drop)

# ---------- 7) 选择 fact 表保留的列
if 'job_id' not in fact_df.columns:
    fact_df['job_id'] = fact_df.index + 1

# 构建最终列清单：优先放主键和外键
final_cols = ['job_id','company_id','job_title_id','salary_id']
# 加入剩余的非维度控制列（比如 category, desc_word_count, skill_flags, salary_min/max 若你想保留）
reserved = [c for c in ['category','desc_word_count','skill_flags','salary_min','salary_max','salary_currency','salary_parsed'] if c in fact_df.columns]
final_cols += reserved

# 如果其他列存在也一并保留（但不重复）
other_cols = [c for c in fact_df.columns if c not in final_cols]
final_cols += other_cols

# 重排
fact_df = fact_df[final_cols]

# ---------- 8) 保存 CSV 文件 ----------
fact_csv = os.path.join(OUT_DIR, "fact_job_postings.csv")
dim_company_csv = os.path.join(OUT_DIR, "dim_company.csv")
dim_job_title_csv = os.path.join(OUT_DIR, "dim_job_title.csv")
dim_salary_csv = os.path.join(OUT_DIR, "dim_salary.csv")

fact_df.to_csv(fact_csv, index=False)
dim_company.to_csv(dim_company_csv, index=False)
dim_job_title.to_csv(dim_job_title_csv, index=False)
dim_salary.to_csv(dim_salary_csv, index=False)

print("Saved files:")
print(" -", fact_csv, "rows:", len(fact_df))
print(" -", dim_company_csv, "rows:", len(dim_company))
print(" -", dim_job_title_csv, "rows:", len(dim_job_title))
print(" -", dim_salary_csv, "rows:", len(dim_salary))

dim_company rows: 165
dim_job_title rows: 1896
dim_salary rows: 182
Missing ids -> company: 0, job_title: 0, salary: 0
Dropping columns from fact: ['post_date', 'post_date_parsed', 'job_description', 'job_type', 'location']
Saved files:
 - data/dwh\fact_job_postings.csv rows: 3644
 - data/dwh\dim_company.csv rows: 165
 - data/dwh\dim_job_title.csv rows: 1896
 - data/dwh\dim_salary.csv rows: 182


In [10]:
display(fact_df.head(20))
display(dim_company.head(5))
display(dim_job_title.head(5))
display(dim_salary.head(5))

,job_id,company_id,job_title_id,salary_id,category,desc_word_count,skill_flags,salary_min,salary_max,salary_currency,salary_parsed,company_name,job_title,salary_offered,post_date_parsed_str,city,country,job_type_std,job_description_clean
0,15cc819a5d9a4b298bcc5fd839bd65a3,1,1,1,Clinical Research,359,"r,biostatistics,clinical",NaN,NaN,None,False,PPD GLOBAL LTD,Senior / Medical Writer (Regulatory),Competitive,2018-04-14,Cambridge,None,Permanent,"As part of our on-going growth, we are current..."
1,31c1a5eac509952eeb8cfdccbeb2fcc4,2,2,1,Science,291,"sas,r,clinical",NaN,NaN,None,False,AL Solutions,Manager of Biometrics,None,2018-04-16,None,Europe,Permanent,Manager of Biometrics – Italy AL Solutions are...
2,7b6abf608611f7c6b97be78b5914d26c,3,3,1,Science,374,r,NaN,NaN,None,False,Seltek Consultants Ltd,Field Service Engineer | Chromatography,None,2018-04-16,None,UK,Permanent,A fantastic opportunity has arisen for an expe...
3,63542ffb0393034e35a3f9753b274b73,4,4,1,Data Management and Statistics,279,"r,pharmacovigilance,clinical",NaN,NaN,None,False,Docs International UK Limited,Data Manager of Project Management,On Application,2018-11-04,M4 Corridor,None,Permanent,Job Details : Utilise extensive clinical data ...
4,7c9de23d13d4dd71f917385947fe5b81,5,5,1,Science,210,r,NaN,NaN,None,False,Hyper Recruitment Solutions Ltd,Strategic Market Analyst,None,2018-04-13,Cambridge,None,Permanent,Hyper Recruitment Solutions are currently look...
5,369628bdd3a511d652223dae4c2ba9ff,6,6,1,Manufacturing & Operations,336,r,NaN,NaN,None,False,BCF Recruitment Ltd,Senior Mechanical Design Engineer,None,2018-10-04,South East,None,Permanent,My client is currently recruiting for a Senior...
6,be5771562626f9f10d0e18acd579c013,7,7,1,Quality-assurance,543,"r,clinical",NaN,NaN,None,False,Austin Fraser,Head of Quality Excellence,"Car allowance, Pension, Bonus, Healthcare etc",2018-12-04,None,Europe,Permanent,Austin Fraser is proud to be supporting one of...
7,7b8ccd6e99b94c17377d0e158cdb9052,5,8,1,Manufacturing & Operations,319,r,NaN,NaN,None,False,Hyper Recruitment Solutions Ltd,Quality Assurance (QA) Document Controller,None,2018-04-14,South East,None,Temporary,We are currently looking for a Quality Assuran...
8,a79f103d858ce81de2d5f01f5ee2ac8f,8,9,1,Clinical Research,849,"r,clinical,gcp",NaN,NaN,None,False,Iqvia Ltd.,Clinical Data Team Lead,Competitive + package,2018-10-04,Manchester,None,Permanent,Due to growth we have exciting opportunities w...
9,2e50da2959c0690e4fbcefedab701823,9,10,1,Medical Information and Pharmacovigilance,327,"r,biostatistics",NaN,NaN,None,False,Barrington James Ltd,SENIOR HEALTH ECONOMIST,Competitive Package,2018-10-04,None,UK,Permanent,I am currently partnered with a rapidly growin...


,company_id,company_name
0,1,PPD GLOBAL LTD
1,2,AL Solutions
2,3,Seltek Consultants Ltd
3,4,Docs International UK Limited
4,5,Hyper Recruitment Solutions Ltd


,job_title_id,job_title
0,1,Senior / Medical Writer (Regulatory)
1,2,Manager of Biometrics
2,3,Field Service Engineer | Chromatography
3,4,Data Manager of Project Management
4,5,Strategic Market Analyst


,salary_id,salary_min,salary_max,salary_currency,salary_parsed
0,1,NaN,NaN,None,False
1,2,60000.0,60000.0,None,True
2,3,80000.0,80000.0,None,True
3,4,95000.0,110000.0,None,True
4,5,75000.0,75000.0,None,True
